In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from qutip import (Qobj, about, basis, coherent, coherent_dm, create, destroy,
                   expect, fock, fock_dm, mesolve, qeye, sigmax, sigmay,
                   sigmaz, tensor, thermal_dm, anim_matrix_histogram,
                   anim_fock_distribution, ket2dm, ptrace)
import qutip
import itertools

from latticefunctions import (purity, purity_rho1, von_neumann_entropy, var_op)


#from master_equation import meq_solve
smesolve = qutip.stochastic.smesolve
ssesolve = qutip.stochastic.ssesolve

In [2]:
N = 7        #Number of particles
L = 7        #Number of sites
J = 1
V0 = 10
periodic_boundary = False

center = int(N/2)    #central site

probed_sites = [1,2,3]
gamma_m = 0*np.ones(len(probed_sites))

#Time
T = 10
N_times = 10

In [3]:
#Defining the chemical potential; may be useful in separate section if U is a more complicated function than a one-site potential
U0 = 0
U_site = 2
U_vec = np.zeros(L)
U_vec[U_site] = U0

In [4]:
#Construction and ordering of the states
basis = [state for state in itertools.product(range(N+1), repeat=L) if sum(state) == N]      
#itertools returns cartesian product (N+1)x(N+1)X... (L times). Then we only take those states with sum of elements equal to N
dim = len(basis)     #dimensionality of the Hilbert space

In [5]:
#Hamiltonian

def basis_index_map(basis):
    """Return a dictionary mapping basis tuples to their indices."""
    return {state: i for i, state in enumerate(basis)}
index_map = basis_index_map(basis)


def kinetic_energy(L, N, J, periodic=False):
    K = np.zeros((dim, dim))

    for i, state in enumerate(basis):
        for n in range(L - 1 + int(periodic)):
            site1 = n
            site2 = (n + 1) % L     #Next site; we can build the opposite transition just by hermitianity

            if state[site1] > 0:
                new_state = list(state)
                new_state[site1] -= 1    #Removing particle from site 1
                new_state[site2] += 1    #Adding particle to site 2
                new_state = tuple(new_state)

                if new_state in index_map:     #If the new state is among the allowed ones. Necessary in case of restricted Hilbert spaces
                    j = index_map[new_state]
                    amplitude = -J*np.sqrt(state[site1] * (state[site2] + 1))    #-J*sqrt(n_1*(n_2+1))
                    K[i, j] += amplitude
                    K[j, i] += amplitude
                    
    return Qobj(K)

def one_particle_potential(U_vec):
    U = np.zeros((dim,dim))
    for i,state in enumerate(basis):
        U[i,i] += sum([state[j]*U_vec[j] for j in range(L)])
    return Qobj(U)

def two_particles_potential(V0):
    V = np.zeros((dim,dim))
    for i,state in enumerate(basis):
        V[i,i] += sum([(V0/2)*state[j]*(state[j]-1) for j in range(L)])
    return Qobj(V)

"""def neighbours(state, periodic=False):
    neighbours_vec = np.zeros(dim)
    state = tuple(state)
    if
    neigh_states.append("""


def particle_move(state, m, n, periodic=False):
    if state[m]>0:
        new_state = list(state)
        new_state[m] -= 1
        new_state[n] += 1
        new_state = tuple(new_state)
        return new_state
    else:
        return None


def current(n):
    j_n = np.zeros((dim,dim), dtype='complex')
    for i, state in enumerate(basis):
        if n!= L-1:
            site1 = n
            site2 = (n + 1)     #Next site; we can build the opposite transition just with the opposite sign

            if state[site1] > 0:
                new_state = list(state)
                new_state[site1] -= 1    #Removing particle from site 1
                new_state[site2] += 1    #Adding particle to site 2
                new_state = tuple(new_state)

                if new_state in index_map:     #If the new state is among the allowed ones. Necessary in case of restricted Hilbert spaces
                    j = index_map[new_state]
                    amplitude = -1j*J*np.sqrt(state[site1] * (state[site2] + 1))    #-iJ*sqrt(n_1*(n_2+1))
                    j_n[i, j] += amplitude
                    j_n[j, i] += -amplitude
        if n!= 0:
            site1 = n
            site2 = (n - 1)     #Next site; we can build the opposite transition just with the opposite sign

            if state[site1] > 0:
                new_state = list(state)
                new_state[site1] -= 1    #Removing particle from site 1
                new_state[site2] += 1    #Adding particle to site 2
                new_state = tuple(new_state)

                if new_state in index_map:     #If the new state is among the allowed ones. Necessary in case of restricted Hilbert spaces
                    j = index_map[new_state]
                    amplitude = -1j*J*np.sqrt(state[site1] * (state[site2] + 1))    #-iJ*sqrt(n_1*(n_2+1))
                    j_n[i, j] += amplitude
                    j_n[j, i] += -amplitude
                
    return Qobj(j_n)
    

            

K = kinetic_energy(L, N, J, periodic=False)
U = one_particle_potential(U_vec)
V = two_particles_potential(V0)

H = K+U+V

ct_current = current(center)
ct_curr_var = var_op(ct_current)

In [6]:
#Site correlation function
def site_corr_function(m,n):
    op = np.zeros((dim, dim), dtype=np.float64)
    
    for i, state in enumerate(basis):
        if state[m]==0:
            continue
        
        new_state = list(state)
        new_state[m] -= 1
        new_state[n] += 1
        new_state = tuple(new_state)

        if new_state in index_map:
            j = index_map[new_state]
            amp = np.sqrt(state[m]) * np.sqrt(state[n] + 1)
            op[j, i] = amp
    return Qobj(op)

In [7]:
dt = T/N_times
tlist = np.arange(0,T,dt)

sc_opss = np.zeros((dim,dim))
for i,state in enumerate(basis):
    for j,site in enumerate(probed_sites):
        sc_opss[i,i] += np.sqrt(gamma_m[j])*state[site]
sc_opss = [Qobj(sc_opss)]

c_opss = []

e_opss = []
for i in range(dim):
    e_opss.append(fock_dm(dim,i))

for n in range(1,L):
    e_opss.append(site_corr_function(0,n))

e_opss.append(ct_current)
e_opss.append(ct_curr_var)

In [ ]:
#Selecting initial state and retrieving its index
idx = index_map[tuple(np.ones(N))]

#Initial state
rho0 = fock(dim,idx)

result = ssesolve(H, rho0, tlist, sc_ops=sc_opss, e_ops=e_opss, ntraj=1)
#result = mesolve(H, rho0, tlist, e_ops = e_opss)

100.0%. Run time:   0.00s. Est. time left: 00:00:00:00


In [ ]:
#Extracting data from the result
basis_exp = np.array(result.expect[0:dim])
occ = np.zeros((L,N_times))     #expected occupation by site
tot = np.zeros((L,N_times))     #verifies total number of particles
for i,state in enumerate(basis):
    for j in range(L):
        occ[j] += state[j]*basis_exp[i]
tot = sum(occ)
Z = sum([occ[i,:]*i for i in range(L)])/N     #center of mass

#Average distance between particles
z = np.zeros(N_times)
for i,state in enumerate(basis):
    for x1 in range(L):
        for x2 in range(x1,L):
            z += basis_exp[i]*state[x1]*state[x2]*np.abs(x1 - x2)
z = 2*z/(N*(N-1))

#Correlation
site_corr = np.zeros((L-1,N_times))
for i in range(0,L-1):
    site_corr[i] = np.array(np.abs(result.expect[dim+i]))
site_corr_avg = np.mean(site_corr, axis=1)
print(site_corr_avg)

In [ ]:
plt.figure()
legend = []
for i in range(L):
    plt.plot(tlist, occ[i])
    legend.append(i)
plt.legend(legend)

In [ ]:
plt.figure()
plt.plot(tlist,Z)

In [ ]:
plt.figure()
plt.plot(tlist,z)

In [ ]:
plt.figure()
plt.plot(np.arange(1,L),site_corr_avg, '.') 

In [ ]:
plt.figure()
center_current = result.expect[dim+L-1]
plt.plot(tlist, center_current)

abs_current = np.abs(center_current)
avg_curr = np.mean(abs_current)
print(avg_curr)

In [ ]:
plt.figure()
center_current_var = result.expect[dim+L]
plt.plot(tlist, center_current_var)